In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
path = os.path.join(os.getcwd(),"data")
df_i = pd.read_csv(os.path.join(path, "articles.csv"))
df_c = pd.read_csv(os.path.join(path, "customers.csv"))
df_t = pd.read_csv(os.path.join(path, "transactions_train.csv"))

In [4]:
### Cleaning df_i ###

# Drop product_code, graphical_appearance_no, colour_group_code, department_no, department_name,
# index_name, section_name, perceived_colour_value_id, perceived_colour_master_id, index_code,
# index_group_no, section_no, garment_group_no

# columns_to_drop_i = ['product_code', 'graphical_appearance_no', 'colour_group_code', 'department_no',
#                      'department_name', 'index_name', 'section_name', 'perceived_colour_value_id',
#                      'perceived_colour_master_id', 'index_code', 'index_group_no', 'section_no',
#                      'garment_group_no']
columns_to_keep_i = ['article_id', 'prod_name', 'product_type_no', 'product_type_name', 'product_group_name',
                     'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name',
                     'perceived_colour_master_name', 'index_group_name', 'garment_group_name', 'detail_desc']
cleaned_df_i = df_i[columns_to_keep_i]

# Replace NaN values in detail_desc with 'Unknown'

cleaned_df_i.fillna('Unknown', inplace=True)
cleaned_df_i['detail_desc'].isnull().any()

# Check cleaning
cleaned_df_i.isnull().sum()

/var/folders/yp/y72nt3x13xxf6y_95ftgm0k00000gn/T/ipykernel_79796/610467727.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df_i.fillna('Unknown', inplace=True)


article_id                      0
prod_name                       0
product_type_no                 0
product_type_name               0
product_group_name              0
graphical_appearance_name       0
colour_group_name               0
perceived_colour_value_name     0
perceived_colour_master_name    0
index_group_name                0
garment_group_name              0
detail_desc                     0
dtype: int64

In [5]:
### Cleaning df_c ###

# Drop FN, Active, club_member_status, postal_code

columns_to_keep = ['customer_id', 'age', 'fashion_news_frequency']
cleaned_df_c = df_c[columns_to_keep]

# Replace “None” with “NONE” in fashion_news_frequency
# Replace NA values in fashion_news_frequency with “NONE”
# Replace NA values in age with mean age

new_value = "NONE"
old_value = 'None'
cleaned_df_c['fashion_news_frequency'].replace(old_value, new_value, inplace=True)
cleaned_df_c['fashion_news_frequency'].fillna(new_value, inplace=True)

mean_age = cleaned_df_c['age'].mean()
cleaned_df_c['age'].fillna(mean_age, inplace=True)


# Check cleaning
cleaned_df_c.head()
cleaned_df_c.isnull().sum()
#Exporting to package
# cleaned_df_i.to_pickle('cleaned_df_i.pkl')

/var/folders/yp/y72nt3x13xxf6y_95ftgm0k00000gn/T/ipykernel_79796/2340333433.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_df_c['fashion_news_frequency'].replace(old_value, new_value, inplace=True)
/var/folders/yp/y72nt3x13xxf6y_95ftgm0k00000gn/T/ipykernel_79796/2340333433.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clea

In [6]:
### Cleaning df_t ###
cleaned_df_t = df_t

# Change t_dat to date type
cleaned_df_t['t_dat'] = pd.to_datetime(cleaned_df_t['t_dat'], format="%Y-%m-%d")


# Check cleaning
print(cleaned_df_t.dtypes)

t_dat               datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object


In [8]:
# Join Articles and Transactions Dataset on article_id
joined_i_t = pd.merge(cleaned_df_t, cleaned_df_i, on='article_id', how='left')
# print(type(joined_i_t))

In [11]:
# Join Customers and Joined Articles and Transaction Dataset on customer_id
joined_df_i_t_c = pd.merge(joined_i_t, cleaned_df_c, on='customer_id', how='outer')

In [8]:
df_t_train = cleaned_df_t[(cleaned_df_t['t_dat'] >= '2019-04-01') & (cleaned_df_t['t_dat'] <= '2019-06-14')]
df_t_val = cleaned_df_t[(cleaned_df_t['t_dat'] >= '2019-06-15') & (cleaned_df_t['t_dat'] <= '2019-06-30')]
df_t_test = cleaned_df_t[(cleaned_df_t['t_dat'] >= '2019-07-01') & (cleaned_df_t['t_dat'] <= '2019-07-31')]

df_c_train = cleaned_df_c[cleaned_df_c['customer_id'].isin(df_t_train['customer_id'])]
df_c_val = cleaned_df_c[cleaned_df_c['customer_id'].isin(df_t_val['customer_id'])]
df_c_test = cleaned_df_c[cleaned_df_c['customer_id'].isin(df_t_test['customer_id'])]

# Exporting Cleaned Datasets

In [12]:
# df_t_train.to_pickle(os.path.join(os.getcwd(),'data','df_t_train.pkl'))
# df_t_val.to_pickle(os.path.join(os.getcwd(),'data', 'df_t_val.pkl'))
# df_t_test.to_pickle(os.path.join(os.getcwd(),'data', 'df_t_test.pkl'))
# df_c_train.to_pickle(os.path.join(os.getcwd(),'data', 'df_c_train.pkl'))
# df_c_val.to_pickle(os.path.join(os.getcwd(),'data', 'df_c_val.pkl'))
# df_c_test.to_pickle(os.path.join(os.getcwd(),'data', 'df_c_test.pkl'))
# cleaned_df_i.to_pickle(os.path.join(os.getcwd(),'data', 'cleaned_df_i.pkl'))
# joined_i_t.to_pickle(os.path.join(os.getcwd(),'data', 'cleaned_data', 'joined_df_i_t.pkl'))
# joined_df_i_t_c.to_pickle(os.path.join(os.getcwd(),'data', 'cleaned_data', 'joined_df_i_t_c'))
